In [15]:
%pip install -e 'src/hear21passt'
%pip install tensorboard

Obtaining file:///root/data/exploratory_notebooks/da/src/hear21passt
  Preparing metadata (setup.py) ... done
  Attempting uninstall: hear21passt
    Found existing installation: hear21passt 0.0.23
    Uninstalling hear21passt-0.0.23:
      Successfully uninstalled hear21passt-0.0.23
  Running setup.py develop for hear21passt
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os._exit(00)

In [3]:
import sys
import os
import datetime
import re

import numpy as np
import pandas as pd
import torch
import torchaudio
import tqdm

from operator import itemgetter
from hashlib import sha1

from torch.optim import AdamW, lr_scheduler
from torch.utils.data import DataLoader, default_collate
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ChainDataset

from sklearn.metrics import f1_score

from IPython.display import Audio

from optim.lr_scheduler import get_scheduler_lambda
from utils.model import get_pretrained_passt_model
from utils.gdsc22_dataset_legacy import BuzzMapDataset, BuzzMapTestDataset
from utils.dataset_augmentations import (
    BuzzMapTransformedDataset, 
    BuzzMapTransformedTestDataset, 
    MixUp1Level,
    MixUp2Level, 
    RandomGain,
    Rolling,
    ColoredNoise,
    NormalizeAmplitudes
)

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
torch.manual_seed(42)
np.random.seed(42)

In [5]:
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
data_processed_dir = os.path.join(project_dir, 'gdsc_data/data_processed/')
data_source_dir = os.path.join(project_dir, 'gdsc_data/data_source/')

In [6]:
ts = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

MODEL_NAME = f"gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_fixed_shuffle_epochs20_transformation=[normed]_epochs20_transformations=[normed; gain; roll; mixup; color_noise]@{ts}.pt"
TORCH_SAVE_PATH = os.path.join(project_dir, f'checkpoints/{MODEL_NAME}')

TENSORBOARD_RUNS_PATH = os.path.join(project_dir, "torch_runlogs/")
TENSORBOARD_EXP_FOLDER = f"PaSST_kd_NOEXP_lr_mismatch_size_t40_f4_fixed_shuffle_epochs20_transformation=[normed]_epochs20_transformations=[normed; gain; roll; mixup; color_noise]@{ts}"
TENSORBOARD_EXP_COMMENT = "PaSST_s_kd_p16_128_ap486_NOEXP_lr_fixed_shuffle_epochs20_transformation=[normed]_epochs20_transformations=[normed; gain; roll; mixup; color_noise]"

TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 32

# fine tuning and ohe of real labels
CLASSIFIER_N_CLASSES = 66

DATA_TAG = "data_8c86715"

DEBUG_TRANSFORMATIONS = True

In [7]:
#final_dataset_path = find_final_dataset_path(data_processed_dir, DATA_TAG)
final_dataset_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '01_apply_ir_function')
final_dataset_val_as_train_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '03_apply_ir_function_on_validation')

final_dataset_path = os.path.join(data_processed_dir, DATA_TAG, '04_split_original_into_windows')
final_dataset_val_as_train_path = os.path.join(data_processed_dir, DATA_TAG, '05_split_resampled_validation_into_windows/')
final_dataset_test_path = os.path.join(data_processed_dir, DATA_TAG, '05_split_resampled_validation_into_windows/')

In [8]:
train_dir_augmented = os.path.join(final_dataset_path_augmented, 'train/')
val_as_train_dir_augmented = os.path.join(final_dataset_val_as_train_path_augmented, 'train/')

train_dir = os.path.join(final_dataset_path, 'train/')
val_as_train_dir = os.path.join(final_dataset_val_as_train_path, 'train/')

val_dir = os.path.join(final_dataset_val_as_train_path, 'val/')
test_dir = os.path.join(final_dataset_test_path, 'test/')

In [9]:
def debug_collate(batch):
    get_1st = itemgetter(0)
    files = ", ".join(map(get_1st, batch)).encode('utf-8')
    worker_info = torch.utils.data.get_worker_info()   
    print(f"Worker_id '{worker_info.id}' batch hash is '{sha1(files).hexdigest()}'") 
    
    batch = default_collate(batch)
    return batch

In [10]:
train_dataset = BuzzMapDataset(
    train_dir,
    metadata_csv=os.path.join(train_dir, 'metadata.csv'), 
    num_classes=CLASSIFIER_N_CLASSES,
)
val_dataset = BuzzMapDataset(
    val_as_train_dir,
    metadata_csv=os.path.join(val_as_train_dir, 'metadata.csv'), 
    num_classes=CLASSIFIER_N_CLASSES
)
test_dataset = BuzzMapTestDataset(
    test_dir,
    metadata_csv=os.path.join(val_as_train_dir, 'metadata.csv'),
    num_classes=CLASSIFIER_N_CLASSES
)

train_dataset_normed = BuzzMapTransformedDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=train_dataset,
    #debug=True
)

train_dataset_transformed = BuzzMapTransformedDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS), 
        #RandomGain(debug=DEBUG_TRANSFORMATIONS), 
        Rolling(debug=DEBUG_TRANSFORMATIONS), 
        #MixUp1Level(debug=DEBUG_TRANSFORMATIONS), 
        ColoredNoise(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=train_dataset,
    #debug=True
)

val_dataset_normed = BuzzMapTransformedDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=val_dataset,
    #debug=True
)

test_dataset_normed = BuzzMapTransformedTestDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=test_dataset,
    #debug=True    
)

# I had too much hope on the IR augmented dataset and took a wrong path...
# train_dataset_augmented = BuzzAugmentedIterableDataset(
#     train_dir_augmented, 
#     metadata_csv=os.path.join(final_dataset_path_augmented, 'metadata_grouped_train.csv'), 
#     shuffle=True,
#     subsample=0.2,
#     total_classes=CLASSIFIER_N_CLASSES
# )
# val_as_train_dataset_augmented = BuzzAugmentedIterableDataset(
#     val_as_train_dir_augmented, 
#     metadata_csv=os.path.join(final_dataset_val_as_train_path_augmented, 'metadata_grouped_train.csv'), 
#     shuffle=True,
#     subsample=0.2,
#     total_classes=CLASSIFIER_N_CLASSES
# )

train_dataloader_normed = DataLoader(
    train_dataset_normed, 
    batch_size=TRAIN_BATCH_SIZE, 
    pin_memory=True,
    shuffle=True,
    num_workers=2,
    prefetch_factor=2,
    worker_init_fn=train_dataset_normed._init_multiprocess_worker,    
)
    
train_dataloader_transformed = DataLoader(
    train_dataset_transformed, 
    batch_size=TRAIN_BATCH_SIZE, 
    pin_memory=True,
    shuffle=True,
    num_workers=2,
    prefetch_factor=2,
    worker_init_fn=train_dataset_transformed._init_multiprocess_worker,
    #collate_fn=debug_collate
)

val_dataloader_normed = DataLoader(
    val_dataset_normed, 
    batch_size=VAL_BATCH_SIZE, 
    pin_memory=True,
    shuffle=True,
    num_workers=2,
    prefetch_factor=2,
    worker_init_fn=train_dataset_normed._init_multiprocess_worker,        
)

In [19]:
sample = next(iter(train_dataset_transformed))

<class 'utils.dataset_augmentations.NormalizeAmplitudes'>: Normalized amplitude '-29.49915885925293' by '9.49915885925293'. Target dBFS='-20'
<class 'utils.dataset_augmentations.Rolling'>: Rolling with interval '-24'
<class 'utils.dataset_augmentations.ColoredNoise'>: noise fun='<bound method ColoredNoise.blue_noise of <utils.dataset_augmentations.ColoredNoise object at 0x7f0b8b3c2aa0>>'; snr='22'; original_SNR='-20.008317947387695', scaled_noise_dBFS=-42.00831604003906


In [20]:
Audio(sample[3].reshape(1, -1), rate=32000, normalize=True)

In [ ]:
20 * torch.log10((sample[3] ** 2).mean())

In [17]:
sample

('Yoyettacelis_GBIF2465208563_IN36000894_50988.wav',
 'Yoyettacelis_GBIF2465208563_IN36000894_50988_1.wav',
 "STEP1:<class 'utils.dataset_augmentations.NormalizeAmplitudes'>: Normalized amplitude '-29.49915885925293' by '9.49915885925293'. Target dBFS='-20';STEP2:<class 'utils.dataset_augmentations.Rolling'>: Rolling shift='-22';STEP3:<class 'utils.dataset_augmentations.MixUp1Level'>: Mixup yes=tensor([False]) ;STEP4:<class 'utils.dataset_augmentations.ColoredNoise'>: noise fun='<bound method ColoredNoise.blue_noise of <utils.dataset_augmentations.ColoredNoise object at 0x7fb375a25d20>>'; snr='33'; original_SNR='-19.97350311279297', scaled_noise_dBFS=-52.97350311279297",
 tensor([-0.0036, -0.0041, -0.0002,  ..., -0.0055, -0.0055, -0.0049]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      